(c) 2024 Manuel Razo. This work is licensed under a [Creative Commons
Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/).
All code contained herein is licensed under an [MIT
license](https://opensource.org/licenses/MIT).

In [69]:
# Import project package
import Antibiotic
import Antibiotic.mh as mh

# Import packages for storing results
import DimensionalData as DD

# Load CairoMakie for plotting
using CairoMakie
import PairPlots
import ColorSchemes

# Import basic math libraries
import StatsBase
import LinearAlgebra
import Random
import Distributions
import Distances

# Activate backend
CairoMakie.activate!()

# Set PBoC Plotting style
Antibiotic.viz.theme_makie!()

# Evolutionary dynamics on a fitness and mutational landscape with hidden dimensions

In this notebook, we will explore the effects of "hidden dimensions" on the evolutionary dynamics of a population. Effectively, this means that we will use the same Metropolis-Hastings algorithm developed previously, but one of the
dimensions in the phenotypic space will not be under selection, giving an extra degree of freedom.

## 3D Example

Let's begin with a simple 3D example. To remove the selection pressure in one 
of the dimensions, we will set the variance of that dimension to a very large value.

In [ ]:
# Evolution condition amplitude
fit_evo_amplitude = 5.0
# Evolution condition mean
fit_evo_mean = [0.0, 0.0, 0.0]
# Evolution condition covariance
fit_evo_cov = [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 100.0]
# Create fitness landscape
fit_evo = mh.GaussianPeak(fit_evo_amplitude, fit_evo_mean, fit_evo_cov)

Let's visualize the fitness landscape as a corner plot, where each panel shows either the marginal distribution or a 2D slice through the landscape.

In [ ]:
# Define ranges of phenotypes to evaluate
x = y = z = range(-6, 6, length=100)
coords = (x, y, z)

# Evaluate fitness landscape
F = mh.fitness(coords, fit_evo)

In [ ]:
# Initialize figure
fig = Figure(size=(200 * length(coords), 200 * length(coords)))

# Add grid layout
gl = fig[1, 1] = GridLayout()

# Extract indexes of axes to form lower triangular grid
axes_idx = [
    idx for idx in CartesianIndices((length(coords), length(coords)))
    if idx[1] >= idx[2]
]

# Extract dimensions
dims = DD.dims(F)

# Add axes to figure
for (i, idx) in enumerate(axes_idx)
    # Extract coordinates for this axis
    local x, y = idx.I
    # Extract dimension for this axis
    xdim = dims[x]
    ydim = dims[y]
    # Add axis to figure
    ax = Axis(gl[x, y], aspect=AxisAspect(1))
    # Hide x and y ticks
    hidedecorations!(ax)

    # Check if x == y
    if x == y
        # Compute marginal fitness landscape by summing over the other dimension
        F_marg = vec(sum(F, dims=dims[[d for d in 1:length(dims) if d != x]]))
        # Plot marginal fitness landscape
        lines!(ax, coords[x], F_marg, color=:black)
    else
        # Marginalize over the other dimension
        F_marg = sum(F, dims=dims[[d for d in 1:length(dims) if d != x && d != y]])
        # Drop dimensions that were marginalized over
        # NOTE: The dims argument must be a tuple of the dimensions to drop.
        F_marg = dropdims(
            F_marg.data,
            dims=tuple(findall(size(F_marg.data) .== 1)...)
        )
        # Plot fitness landscape
        heatmap!(ax, coords[x], coords[y], F_marg, colormap=:algae)
        contour!(ax, coords[x], coords[y], F_marg, color=:white)
    end
end

# Reduce spacing between subplots
rowgap!(gl, 5)
colgap!(gl, 5)

fig